In [1]:
%load_ext autoreload
%autoreload 2

from util import *
import pandas as pd

In [2]:
def test_deadline(all_orders, rider, shop_seq, dlv_seq):
    pickup_times, dlv_times = get_pd_times(all_orders, rider, shop_seq, dlv_seq)

    for k, dlv_time in dlv_times.items():
        return all_orders[k].deadline - dlv_time

In [3]:
import os

vol_arr_dict = {"BIKE" :[], "CAR": [], "WALK": []}
tw_arr_dict = {"BIKE" :[], "CAR": [], "WALK": []}
label_arr_dict = {"BIKE" :[], "CAR": [], "WALK": []}

prob_dir = "alg_test_problems"
weight_dir = "weight"

for file_name in os.listdir(weight_dir):

    weight_file = os.path.join(weight_dir, file_name)
    problem_file = os.path.join(prob_dir, file_name[7:])

    with open(weight_file, 'r') as f:
        weight = json.load(f)

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time)


    for key, value in weight.items():
        # label_mat_dict[key] = pd.DataFrame(value) # label
        label_arr_dict[key] += np.array(value).flatten().tolist()

    # 어차피 세번만 도니까 따로 두자
    for rider in ALL_RIDERS:
        volume_matrix = []

        for order1 in ALL_ORDERS:
            volume_matrix.append(
                [rider.capa - (order1.volume + order2.volume) for order2 in ALL_ORDERS]
                )   
            
        vol_arr_dict[rider.type] += np.array(volume_matrix).flatten().tolist()

    
    for rider in ALL_RIDERS:
        tw_matrix = []

        for order1 in ALL_ORDERS:
            li = []
            for order2 in ALL_ORDERS:
                merged_orders = [order1.id, order2.id]
                max_gap = -10000 
                for shop_pem in permutations(merged_orders):
                    for dlv_pem in permutations(merged_orders):
                        gap = test_deadline(ALL_ORDERS, rider, shop_pem, dlv_pem)

                        if gap > max_gap:
                            max_gap = gap
                
                li.append(max_gap)

            tw_matrix.append(li)

        tw_arr_dict[rider.type] += np.array(tw_matrix).flatten().tolist()

print(len(vol_arr_dict['BIKE']))
print(len(label_arr_dict['BIKE']))
print(len(tw_arr_dict["BIKE"]))

105000
105000
105000


In [4]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as met

rider = 'BIKE'
data = {'vol': vol_arr_dict[rider], 'tw': tw_arr_dict[rider] }

X = pd.DataFrame(data)
y = label_arr_dict[rider]

# 학습 및 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # DMatrix 객체 생성
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# XGBoost 모델 설정
params = {
    'max_depth': 3,
    'eta': 0.1,
    'objective': 'reg:absoluteerror',
    'eval_metric': 'mae',
}

num_round = 50
bst = xgb.train(params, dtrain, num_round)

# 예측
preds = bst.predict(dtest)

# 정확도 계산
mse = met(y_test, preds)
print(f"MAE: {mse:.2f}")

MAE: 0.04


In [7]:
nd  = {'vol':[10,20,30], 'tw':[800,900,1000]}
bst.predict(xgb.DMatrix(pd.DataFrame(nd)))

array([1., 1., 1.], dtype=float32)

In [11]:
np.array([1,2,3,4]).reshape(2,2).tolist()

[[1, 2], [3, 4]]

In [28]:
import pickle
filename = 'xgb_v_t.model'

# 모델 저장
pickle.dump(bst, open(filename, 'wb'))

# 모델 불러오기
# xgb_model = pickle.load(open(filename, 'rb'))

In [26]:
ALL_ORDERS[0].__dict__.keys()

dict_keys(['id', 'order_time', 'shop_lat', 'shop_lon', 'dlv_lat', 'dlv_lon', 'cook_time', 'volume', 'deadline', 'ready_time'])

In [65]:
ALL_RIDERS[0].__dict__

{'type': 'BIKE',
 'speed': 5.291005291005291,
 'capa': 100,
 'var_cost': 60,
 'fixed_cost': 8000,
 'service_time': 120,
 'available_number': 20,
 'T': array([[ 120.,  809., 1093., ..., 1703.,  784.,  730.],
        [ 809.,  120.,  898., ..., 1386.,  548.,  709.],
        [1093.,  898.,  120., ...,  731.,  498., 1403.],
        ...,
        [1703., 1386.,  731., ...,  120., 1072., 1952.],
        [ 784.,  548.,  498., ..., 1072.,  120., 1025.],
        [ 730.,  709., 1403., ..., 1952., 1025.,  120.]])}

In [44]:
import pandas as pd

ord_df = pd.DataFrame(columns=ALL_ORDERS[0].__dict__.keys())

for row in ALL_ORDERS:
    ord_df = pd.concat([ord_df, pd.DataFrame([vars(row)])])


ord_df

/var/folders/3l/z2_cjhyd20v1_yppby93tmnh0000gn/T/ipykernel_89132/4223044299.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ord_df = pd.concat([ord_df, pd.DataFrame([vars(row)])])


,id,order_time,shop_lat,shop_lon,dlv_lat,dlv_lon,cook_time,volume,deadline,ready_time
0,0,28,35.997628,125.982584,35.995128,125.977870,600,21,2176,628
0,1,86,36.012988,126.004387,36.004457,126.004163,900,12,2680,986
0,2,118,35.991074,126.022580,35.984552,126.029739,600,22,2419,718
0,3,155,36.021098,126.019330,36.016537,126.019030,600,44,2303,755
0,4,156,36.018153,125.972029,36.017555,125.986989,900,16,2882,1056
...,...,...,...,...,...,...,...,...,...,...
0,95,3466,35.991851,125.992472,35.988031,125.993688,600,13,5591,4066
0,96,3474,36.016543,125.996638,36.018567,125.998289,900,28,5843,4374
0,97,3493,35.990046,126.040079,35.988455,126.048018,600,27,5717,4093
0,98,3496,36.001260,126.004988,35.999274,126.010367,1200,33,6252,4696
